In [1]:
#Typical imports for data analysis
from IPython import display
import math
import pandas as pd
import numpy as np
import nltk


from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords

In [7]:
#Uses praw to scrape the headlines from a given subreddit
def scraper(sreddit):
    hlines=set()
    for submission in reddit.subreddit(sreddit).top(limit=None):
        hlines.add(submission.title)
        display.clear_output()
        print(len(hlines))
    return hlines

#Assigns a sentiment value from -1 to 1 to a list of text
def sentimentAnalysis(textList):
    sia = SIA()
    results = []

    for line in textList:
        pol_score = sia.polarity_scores(line)
        pol_score['headline'] = line
        results.append(pol_score)
    
    return results

#Writes a csv file for each subreddit, with each headline and a rounded score (positive, negative, neutral)
#This function also writes the positive and negative scores each subreddit got
#Should rewrite this.
def csvOutput(multiArray,name):
    df=pd.DataFrame.from_records(multiArray)
    df['label'] = 0
    df.loc[df['compound'] > 0.2, 'label'] = 1
    df.loc[df['compound'] < -0.2, 'label'] = -1
    
    df2 = df[['headline', 'label']]
    df2.to_csv('headlines_'+ name +'.csv', mode='a', encoding='utf-8', index=False)
    
    print(df.label.value_counts())

    print(df.label.value_counts(normalize=True) * 100)
    
    test=(df.label.value_counts(normalize=True) * 100).tolist()
    poslist.append(test[1])
    neglist.append(test[2])
    
    tokenCount(df)
    
#Tokenizes the words in each headline, barring stop words
def processText(headlines):
    tokens = []
    for line in headlines:
        toks = tokenizer.tokenize(line)
        toks = [t.lower() for t in toks if t.lower() not in stop_words]
        tokens.extend(toks)
    
    return tokens

#Finds the words most commonly used in positive and negative headlines
def tokenCount(df):
    pos_lines = list(df[df.label == 1].headline)

    pos_tokens = processText(pos_lines)
    pos_freq = nltk.FreqDist(pos_tokens)

    pwordslist.append(pos_freq.most_common(3))
    
    neg_lines = list(df[df.label == -1].headline)

    neg_tokens = processText(neg_lines)
    neg_freq = nltk.FreqDist(neg_tokens)

    nwordslist.append(neg_freq.most_common(3))

In [8]:
import praw
#In order to use this code, replace the placeholders with the data given by reddit
reddit = praw.Reddit(client_id='your-client-id',
                     client_secret='your-client-secret',
                     user_agent='your-reddit-username')

#Some test subs, including teams from the EPL, NFL, NBA
subs=['nfl','nba']
#subs=['AFCBournemouth', 'Gunners', 'BrightonHoveAlbion', 'burnley', 'bluebirds', 'chelseafc', 'crystalpalace', 'Everton', 'fulhamfc', 'HuddersfieldTownFC', 'lcfc', 'LiverpoolFC', 'MCFC', 'reddevils', 'NUFC', 'SaintsFC', 'coys', 'Watford_FC', 'WWFC', 'Hammers']
#subs=["ravens","steelers","browns","bengals","patriots","miamidolphins","buffalobills","nyjets","chibears","minnesotavikings","greenbaypackers","detroitlions","cowboys","eagles","redskins","nygiants","texans","colts","tennesseetitans","jaguars","kansascitychiefs","chargers","denverbroncos","oaklandraiders","saints","falcons","panthers","buccaneers","losangelesrams","seahawks","49ers","azcardinals"]
#subs=["mavericks","denvernuggets","warriors","rockets","laclippers","lakers","memphisgrizzlies","timberwolves","nolapelicans","thunder","suns","ripcity","kings","nbaspurs","utahjazz","nba","atlantahawks","bostonceltics","gonets","charlottehornets","chicagobulls","clevelandcavs","detroitpistons","pacers","heat","mkebucks","nyknicks","orlandomagic","sixers","torontoraptors","washingtonwizards"]
poslist=[]
neglist=[]
pwordslist=[]
nwordslist=[]


tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')


for club in subs:
    headlines = scraper(club)
    sentiments = sentimentAnalysis(headlines)
    csvOutput(sentiments,club)
    
    
print(poslist)
print(neglist)
print(pwordslist)
print(nwordslist)

995
 0    450
 1    342
-1    203
Name: label, dtype: int64
 0    45.226131
 1    34.371859
-1    20.402010
Name: label, dtype: float64
this is test
[45.22613065326633, 34.37185929648241, 20.402010050251256]
[35.07537688442211, 34.37185929648241]
[35.07537688442211, 34.37185929648241]
[21.20603015075377, 20.402010050251256]
[[('nfl', 62), ('game', 58), ('like', 38)], [('nba', 61), ('lebron', 59), ('game', 44)]]
[[('nfl', 35), ('game', 23), ('team', 16)], [('game', 30), ('lebron', 22), ('nba', 20)]]


In [5]:
#Gathers the outputs and presents it as a dataframe
test=[]
for i in range(len(subs)):
    test.append([subs[i],poslist[i],neglist[i],poslist[i]-neglist[i],pwordslist[i],nwordslist[i]])
outputdf=pd.DataFrame(test, columns=['Sub','Positive %', "Negative %","Difference","Positive Words", "Negative Words"], index=range(1,len(subs)+1))

In [6]:
pholder=outputdf.sort_values(by="Difference", ascending=False)
pholder.reset_index(drop=True, inplace=True)
pholder

,Sub,Positive %,Negative %,Difference,Positive Words,Negative Words
0,nba,34.371859,20.40201,13.969849,"[(nba, 61), (lebron, 59), (game, 44)]","[(game, 30), (lebron, 22), (nba, 20)]"
1,nfl,35.075377,21.20603,13.869347,"[(nfl, 62), (game, 58), (like, 38)]","[(nfl, 35), (game, 23), (team, 16)]"
